In [9]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import seaborn as sbn
import sklearn.metrics as sm
import pickle
from sklearn.naive_bayes import GaussianNB as nb
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as lr
import warnings
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.callbacks import *
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

# Tắt cảnh báo
warnings.filterwarnings("ignore")

In [10]:
for i in os.listdir('./OBJcropped/'):
    img = cv2.imread(f'./OBJcropped/{i}')
    img = cv2.resize(img,(50,50))
    try:
        os.chdir('./cnn_dataset')
    except:
        os.mkdir('./cnn_dataset')
        os.chdir('./cnn_dataset')
    cv2.imwrite(f'{i}', img)
    os.chdir('..')

In [11]:
# Đọc tất cả các ảnh từ một folder
# Resize ảnh về (100,100)
def load_images(folder):
    train_data=[]
    for label in os.listdir(folder):
        print(label, " Started!")
        path=folder+'/'+label
        for img in os.listdir(path):
            img=cv2.imread(path+'/'+img)
            img=cv2.resize(img,(50,50))
            # new_img=cv2.resize(img,(600,100))
            if img is not None:
                train_data.append([img,label])
        print(label, " ended!")
    return train_data

# Load ảnh train và label

path='./CNN/train'
# path='./Saved Dataset'
train_data=load_images(path)


# Load ảnh test và label

path='./CNN/test'
# path='./Test'
test_data=load_images(path)

# Trộn các data trong tập train và tập test
random.shuffle(train_data)
random.shuffle(test_data)

# Phân chia dữ liệu thành tập train và test
train_images=[]
train_labels=[]
test_images=[]
test_labels=[]

for feature, label in train_data:
    train_images.append(feature)
    train_labels.append(label)

for feature, label in test_data:
    test_images.append(feature)
    test_labels.append(label)

# Chuyển mảng ảnh sang ma trận numpy
train_images=np.array(train_images)
test_images=np.array(test_images)

# train_images=train_images.reshape((-1,50, 50,1))
# test_images=test_images.reshape((-1,50, 50,1))

# Chuyển datatype sang float32 và chuẩn hóa dữ liệu

train_images= train_images.astype('float32')
test_images = test_images.astype('float32')

train_images=train_images/255.0
test_images=test_images/255.0

# Chuyển đổi các label về dạng số

le=LabelEncoder()
le.fit_transform(train_labels)
le.fit_transform(test_labels)
train_labels_label_encoded=le.transform(train_labels)
test_labels_label_encoded=le.transform(test_labels)

# Chuyển label vể one hot vector

train_labels_one_hot = to_categorical(train_labels_label_encoded)
test_labels_one_hot = to_categorical(test_labels_label_encoded)

greenlight  Started!
greenlight  ended!
left  Started!
left  ended!
noleft  Started!
noleft  ended!
noright  Started!
noright  ended!
nostraight  Started!
nostraight  ended!
redlight  Started!
redlight  ended!
right  Started!
right  ended!
stop  Started!
stop  ended!
straight  Started!
straight  ended!
yellowlight  Started!
yellowlight  ended!
greenlight  Started!
greenlight  ended!
left  Started!
left  ended!
noleft  Started!
noleft  ended!
noright  Started!
noright  ended!
nostraight  Started!
nostraight  ended!
redlight  Started!
redlight  ended!
right  Started!
right  ended!
stop  Started!
stop  ended!
straight  Started!
straight  ended!
yellowlight  Started!
yellowlight  ended!


In [12]:
print(train_images.shape)
test_images.shape

(425, 50, 50, 3)


(253, 50, 50, 3)

In [13]:
input_shape=(50,50,3)
n_classes=10

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

model=create_model()
batch_size=256
epochs=200
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# In ra thông tin các layer
model.summary()

datagen = ImageDataGenerator(channel_shift_range=0.2)

# history = model.fit(train_images, train_labels_one_hot, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(test_images, test_labels_one_hot))
# model.evaluate(test_images, test_labels_one_hot)
earlystop = EarlyStopping(patience=10)   
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',   #điều cần quan tâm là độ chính xác nếu độ chính xác 
                                                                 #không tăng thì lr sẽ giảm
                                            patience=3,          #số lượng epoch mà sau đó không cải thiện chương trình sẽ stop
                                            verbose=1,           #có cập nhật và hiển thị trạng thái khi reduce
                                            factor=0.5,          #new_lr=factor*lr
                                            min_lr=1e-6)      #giới hạn lr
checkpoint = ModelCheckpoint('CNN_models/CNN-model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
callb=[learning_rate_reduction,checkpoint]

# results = model.fit(X_train,y_train, steps_per_epoch=len(X_train)/16, epochs=100, verbose=1,batch_size=16, validation_data=(X_val, y_val), callbacks=callb)
model.fit_generator(datagen.flow(train_images, train_labels_one_hot, batch_size=16), steps_per_epoch=len(train_images)/16,
    epochs=200, verbose=1, validation_data=(test_images, test_labels_one_hot), callbacks=callb)

# Plot hàm loss

# plt.figure(figsize=[8,6])
# plt.plot(history.history['loss'],'r',linewidth=2.0)
# plt.plot(history.history['val_loss'],'b',linewidth=2.0)
# plt.legend(['Training loss', 'Validation Loss'],fontsize=15)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Loss',fontsize=16)
# plt.title('Loss Curves',fontsize=16)

# # Plot accuracy

# plt.figure(figsize=[8,6])
# plt.plot(history.history['accuracy'],'r',linewidth=2.0)
# plt.plot(history.history['val_accuracy'],'b',linewidth=2.0)
# plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=15)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Accuracy',fontsize=16)
# plt.title('Accuracy Curves',fontsize=16)

# # Lưu lại giá trị trọng số của mô hình
# model.save_weights('./Saved_Files/CNN.h5')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 50, 50, 32)        896       
                                                                 
 conv2d_29 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 24, 24, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_11 (Dropout)        (None, 24, 24, 32)        0         
                                                                 
 conv2d_30 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 conv2d_31 (Conv2D)          (None, 22, 22, 32)        9248      
                                                      